# Intent Classification using LSTM 
This use-case provides a demo of how LSTM can be used for Intent classification in texts. 

## Workflow:

1.   Understanding the problem
2.   Reading the data and understanding it
3.   Data Preprocessing
4.   Build LSTM model
5.   Train & Evaluate the model

##1. Understanding the problem
Intent Classification is the automated association of text to a specific intention. For example: Let's say you are writing an email to one of the Airlines and the text of the same is 'Can you please cancel my ticket with PNR 123456'. The intent of the customer here is 'Cancellation of Air Ticket'.

The idea of this use case to introduce the concept of Intent classification and how can LSTM be used to solve this. 

###Import the necessary libraries
Please load the following packages before you proceed further. 


In [ ]:
import numpy as np 
import pandas as pd 
import nltk

from sklearn.preprocessing import OneHotEncoder as oneHot
from nltk.corpus import stopwords
from nltk import word_tokenize
from string import punctuation
from nltk.stem import PorterStemmer
nltk.download('punkt')
nltk.download('stopwords')  

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.layers import BatchNormalization, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy as cce
from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.metrics import AUC
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential

from sklearn.metrics import classification_report



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


##2. Collecting the Data
The ATIS(Air Travel Information System) data is a rich corpus that contains natural language text used by general public to book flight tickets, enquire about flight timings, prices etc. 

The Train and test data can be downloaded from https://www.kaggle.com/hassanamin/atis-airlinetravelinformationsystem

There are 2 columns in each of the above datasets. First column is 'target' which is the output we will be classifying and second column is 'text' which is the user input asking for queries related to flights. 

Basically 'target' is the intent of the customer. 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Read the train and test datasets with column names as target and text
train= pd.read_csv('/content/drive/MyDrive/atis_intents_train.csv',
                       names= ["target", "text"])

test= pd.read_csv('/content/drive/MyDrive/atis_intents_test.csv',
                       names= ["target", "text"])

In [ ]:
print(train.shape, test.shape)

(4834, 2) (800, 2)


####This is how the data looks like

In [ ]:
train.head(10) #Get Top 10 rows from train dataset

,target,text
0,atis_flight,i want to fly from boston at 838 am and arriv...
1,atis_flight,what flights are available from pittsburgh to...
2,atis_flight_time,what is the arrival time in san francisco for...
3,atis_airfare,cheapest airfare from tacoma to orlando
4,atis_airfare,round trip fares from pittsburgh to philadelp...
5,atis_flight,i need a flight tomorrow from columbus to min...
6,atis_aircraft,what kind of aircraft is used on a flight fro...
7,atis_flight,show me the flights from pittsburgh to los an...
8,atis_flight,all flights from boston to washington
9,atis_ground_service,what kind of ground transportation is availab...


 Check the number of intents.

In [ ]:
train['target'].value_counts() #Get counts of different types of target variable in train data. We will not be using this anywhere but it is just for the overview of the data.  

atis_flight            3666
atis_airfare            423
atis_ground_service     255
atis_airline            157
atis_abbreviation       147
atis_aircraft            81
atis_flight_time         54
atis_quantity            51
Name: target, dtype: int64

In [ ]:
train['target'].unique()
#So, we have 8 intents according to the ouput displayed

array(['atis_flight', 'atis_flight_time', 'atis_airfare', 'atis_aircraft',
       'atis_ground_service', 'atis_airline', 'atis_abbreviation',
       'atis_quantity'], dtype=object)

### 3. Preprocessing the Data
We will be doing the following preprocessing steps to get the desired format of the data. 

1. Perform One Hot Encoding on the target variable of both train & test datasets.
2. Convert the text into lower case.
3. Tokenize the words.
4. Remove stop words.
5. Perform stemming & normalization.
6. Convert texts into sequences.
7. Pad the sequences.


In [ ]:
mtarray = np.array(train['target'])
mtarray.reshape(-1,1).shape

(4834, 1)

In [ ]:
mtarray[:5]

array(['atis_flight', 'atis_flight', 'atis_flight_time', 'atis_airfare',
       'atis_airfare'], dtype=object)

In [ ]:
encode_target= oneHot().fit(np.array(train.target).reshape(-1,1)) #We perform one hot encoding on the target variable to convert into a matrix of 0s and 1s. 

type(encode_target)

sklearn.preprocessing._encoders.OneHotEncoder

reshape(-1,1)---> -1 means, dont change the index i.e. 4834 comes as it is. 1 means, add index. So, the output looks like (4834,1)

Perform One Hot Encoding on the target variable. The output of this step would be an array with 0s and 1s. 

In [ ]:
mtarray[:5]

array(['atis_flight', 'atis_flight', 'atis_flight_time', 'atis_airfare',
       'atis_airfare'], dtype=object)

In [ ]:
train_target_encoded= encode_target.transform(np.array(train.target).reshape(-1,1)).toarray()
test_target_encoded= encode_target.transform(np.array(test.target).reshape(-1,1)).toarray()
train_target_encoded[:5]

array([[0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0.]])

Convert text to lower case

In [ ]:
train["text"]= train.text.map(lambda l: l.lower())
test["text"]= test.text.map(lambda l: l.lower())

Next step is to tokenize the text. We use word_tokenize function from nltk library for this purpose. 

In [ ]:
train["text"]= train.text.map(word_tokenize)
test["text"]= test.text.map(word_tokenize)

In [ ]:
#Output of the above exercise looks like this
train["text"]

0       [i, want, to, fly, from, boston, at, 838, am, ...
1       [what, flights, are, available, from, pittsbur...
2       [what, is, the, arrival, time, in, san, franci...
3          [cheapest, airfare, from, tacoma, to, orlando]
4       [round, trip, fares, from, pittsburgh, to, phi...
                              ...                        
4829    [what, is, the, airfare, for, flights, from, d...
4830    [do, you, have, any, flights, from, denver, to...
4831    [which, airlines, fly, into, and, out, of, den...
4832    [does, continental, fly, from, boston, to, san...
4833    [is, there, a, delta, flight, from, denver, to...
Name: text, Length: 4834, dtype: object

Eliminate stop words. 'english' dictionary from nltk.corpus library is used for this purpose. We also remove punctuation along with the removal of stop words.

In [ ]:
def clean_data_rm_stop(strings, stop_list):
    sw= [str for str in strings if str not in stop_list]
    return sw

stop_words= stopwords.words("english")
rm_punc_stop= list(set(punctuation))+ stop_words #Remove punctuation and stop words

train["text"]= train.text.map(lambda tokenlist: clean_data_rm_stop(tokenlist, rm_punc_stop))
test["text"]= test.text.map(lambda tokenlist: clean_data_rm_stop(tokenlist, rm_punc_stop))

In [ ]:
train['text'].head()

0    [want, fly, boston, 838, arrive, denver, 1110,...
1    [flights, available, pittsburgh, baltimore, th...
2    [arrival, time, san, francisco, 755, flight, l...
3                 [cheapest, airfare, tacoma, orlando]
4    [round, trip, fares, pittsburgh, philadelphia,...
Name: text, dtype: object

Stemming & Normalizing


*   Stemming helps in reducing the word to the root form.
*   Normalizing is the process of transforming text into a standard form. Eg: Gud will be converted to good etc.




In [ ]:
def normalize(text):
    return " ".join(text)

#We use PorterStemmer function from nltk.stem library.
stem_func= PorterStemmer()

train["text"]= train.text.map(lambda s: [stem_func.stem(x) for x in s])
train["text"]= train.text.apply(normalize)

test["text"]= test.text.map(lambda s: [stem_func.stem(x) for x in s])
test["text"]= test.text.apply(normalize)

In [ ]:
train['text'].head()

0           want fli boston 838 arriv denver 1110 morn
1       flight avail pittsburgh baltimor thursday morn
2    arriv time san francisco 755 flight leav washi...
3                       cheapest airfar tacoma orlando
4    round trip fare pittsburgh philadelphia 1000 d...
Name: text, dtype: object

Tokenize

In [ ]:
# We use Tokenizer from tensorflow.keras.preprocessing.text library
num_words=10000
text_tokenizer= Tokenizer(num_words)
text_tokenizer.fit_on_texts(train.text) #fit_on_texts - creates the vocabulary index based on word frequency.

tokenized_train_data= text_tokenizer.texts_to_sequences(train.text) #Converting texts to sequences
tokenized_test_data= text_tokenizer.texts_to_sequences(test.text)

type(tokenized_train_data), tokenized_train_data[:5]

(list,
 [[42, 19, 2, 296, 17, 5, 376, 21],
  [1, 37, 8, 10, 45, 21],
  [17, 102, 4, 6, 377, 1, 12, 18],
  [32, 212, 142, 91],
  [36, 30, 16, 8, 11, 180, 134]])

Then, we pad the sequences

In [ ]:
#We use pad_sequences from tensorflow.keras.preprocessing.sequence library
train_data= pad_sequences(tokenized_train_data, maxlen= 20, padding= "pre")
test_data= pad_sequences(tokenized_test_data, maxlen= 20, padding= "pre")
train_data[:5]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  42,
         19,   2, 296,  17,   5, 376,  21],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   1,  37,   8,  10,  45,  21],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,  17,
        102,   4,   6, 377,   1,  12,  18],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,  32, 212, 142,  91],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         36,  30,  16,   8,  11, 180, 134]], dtype=int32)

Let's build a 3 dim array. The dimensions are samples, steps and unique words.



*   batch size

*   time_steps
*   feature


In [ ]:
 text_tokenizer.word_index.keys().__len__()
 #number of unique words

654

In [ ]:
def transform_matrix(data, tokenizer):
    output_shape_mat= [data.shape[0],
                  data.shape[1],
                  tokenizer.word_index.keys().__len__()] #Three dimensional matrix with samples, steps and number of uniques words as each dimension.
    results_data= np.zeros(output_shape_mat) #creates new array with given dimensions.
    
    for i in range(data.shape[0]):
        for j in range(data.shape[1]):
            results_data[i, j, data[i,j]-1]= 1 # In this for loop, we are looping over the shape of the training & test data and assigning the cell of above created zero matrix to 1. We are performing encoding on the unique words to obtain the transformation matrix
    return results_data

trans_matrix_train= transform_matrix(train_data, text_tokenizer) #This will be the matrix on which the lstm model is applied
trans_matrix_test= transform_matrix(test_data, text_tokenizer) #This will be the matrix on which the model is tested

In [ ]:
print(train_data.shape)
print(trans_matrix_train.shape)
print(train['text'][3])
print(train_data[3, :])
print(trans_matrix_train[3,16,:]) #at 31st position there will be one and rest all zero

(4834, 20)
(4834, 20, 654)
cheapest airfar tacoma orlando
[  0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0  32 212
 142  91]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0

### 4. Build LSTM Model

Build a class lstm_model_class that has three methods(methods are similar to functions) in a class, they are:



1.   Build a lstm model 
2.   Train the created lstm model on the train data
3.   Predict the output on the train data

Note: Building a Class with 3 methods helps in tying all these three functions to the same object at the same instance. 

Process of Building a LSTM Model:



1.   Build an embedded layer with dimensions as number of steps and input dimensions. 
2.   Build an LSTM layer with number of steps equal to memory units.  
3.   Then, build a dense layer which is fully connected layer that represents a matrix vector multiplication. 
4.   Apply the relu function after normalization and scaling of the activations. This is the standard activation function used. 
5.   Finally, build the output layer.

Note: 
1. activation function for multi-class classification problem - softmax
2. loss function is categorical cross entropy. 
3. performace metric - Area under the curve(AUC)
4. optimizer would be the Adam optimizer. 





In [ ]:
n_steps = trans_matrix_train.shape[1]
n_features = trans_matrix_train.shape[2]
output_shape = train_target_encoded.shape[1]
print(n_steps, n_features, output_shape)

20 654 8


In [ ]:
#check this code with tf2

n_steps = trans_matrix_train.shape[1]
n_features = trans_matrix_train.shape[2]
output_shape = train_target_encoded.shape[1]

model = Sequential()
model.add(tf.keras.layers.LSTM(units=n_steps, activation='relu', input_shape=(n_steps,n_features)))

model.add(Dense(output_shape))
model.add(BatchNormalization())

model.add(Dense(output_shape, activation='softmax'))
model.compile(optimizer='adam', loss=cce(), metrics=[AUC()])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 20)                54000     
                                                                 
 dense (Dense)               (None, 8)                 168       
                                                                 
 batch_normalization (BatchN  (None, 8)                32        
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 8)                 72        
                                                                 
Total params: 54,272
Trainable params: 54,256
Non-trainable params: 16
_________________________________________________________________


In [ ]:
model.fit(trans_matrix_train, train_target_encoded, validation_split=0.2, epochs=60)

Epoch 1/60
121/121 [==============================] - 14s 58ms/step - loss: 1.6991 - auc: 0.8063 - val_loss: 1.5425 - val_auc: 0.9326
Epoch 2/60
121/121 [==============================] - 12s 96ms/step - loss: 0.7268 - auc: 0.9741 - val_loss: 0.6632 - val_auc: 0.9924
Epoch 3/60
121/121 [==============================] - 12s 96ms/step - loss: 0.3443 - auc: 0.9939 - val_loss: 0.2757 - val_auc: 0.9970
Epoch 4/60
121/121 [==============================] - 12s 97ms/step - loss: 0.2030 - auc: 0.9979 - val_loss: 0.1712 - val_auc: 0.9979
Epoch 5/60
121/121 [==============================] - 6s 50ms/step - loss: 0.1424 - auc: 0.9987 - val_loss: 0.1973 - val_auc: 0.9971
Epoch 6/60
121/121 [==============================] - 5s 42ms/step - loss: 0.1092 - auc: 0.9990 - val_loss: 0.1901 - val_auc: 0.9951
Epoch 7/60
121/121 [==============================] - 4s 37ms/step - loss: 0.0761 - auc: 0.9995 - val_loss: 0.1073 - val_auc: 0.9976
Epoch 8/60
121/121 [==============================] - 6s 53ms/ste

In [ ]:
pred_train = encode_target.inverse_transform(model.predict(trans_matrix_train))
print(classification_report(train.target, pred_train))

152/152 [==============================] - 1s 5ms/step
                     precision    recall  f1-score   support

  atis_abbreviation       0.99      1.00      1.00       147
      atis_aircraft       0.98      1.00      0.99        81
       atis_airfare       0.99      1.00      0.99       423
       atis_airline       0.99      0.99      0.99       157
        atis_flight       1.00      1.00      1.00      3666
   atis_flight_time       0.98      1.00      0.99        54
atis_ground_service       1.00      1.00      1.00       255
      atis_quantity       1.00      1.00      1.00        51

           accuracy                           1.00      4834
          macro avg       0.99      1.00      0.99      4834
       weighted avg       1.00      1.00      1.00      4834



F1 and weighted avg are excellent. We can now move to implement this model on test data and see how it is performing.

In [ ]:
pred_test = encode_target.inverse_transform(model.predict(trans_matrix_test))
print(classification_report(test.target, pred_test))

25/25 [==============================] - 0s 6ms/step
                     precision    recall  f1-score   support

  atis_abbreviation       1.00      1.00      1.00        33
      atis_aircraft       0.78      0.78      0.78         9
       atis_airfare       0.98      1.00      0.99        48
       atis_airline       1.00      0.95      0.97        38
        atis_flight       0.99      0.99      0.99       632
   atis_flight_time       1.00      1.00      1.00         1
atis_ground_service       1.00      0.94      0.97        36
      atis_quantity       0.43      1.00      0.60         3

           accuracy                           0.98       800
          macro avg       0.90      0.96      0.91       800
       weighted avg       0.99      0.98      0.98       800



F1 and weighted avg are excellent. We can settle with this model. 

**Check**

In [ ]:
test[:5]

,target,text
0,atis_flight,would like find flight charlott la vega make s...
1,atis_airfare,april first need ticket tacoma san jose depart 7
2,atis_flight,april first need flight go phoenix san diego
3,atis_flight,would like flight travel one way phoenix san d...
4,atis_flight,would like flight orlando salt lake citi april...


In [ ]:
pred_test[:5]

array([['atis_flight'],
       ['atis_airfare'],
       ['atis_flight'],
       ['atis_flight'],
       ['atis_flight']], dtype=object)